In [8]:
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline
import random

## 2 -Gait Modeal:  LSTMs in Keras: 

 LSTM model that takes as inputs my gait angles features.


In [1]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
import tensorflow as tf
np.random.seed(1)

Using TensorFlow backend.


In [9]:
# load data with Pandas
import pandas as pd
X_all = pd.read_csv("cycles_for_LSTM/X_all_10_66.csv")
# print things about the files
print(X_all.shape)
#print(X_train.head(1))

(486, 662)


In [17]:
def Reshape_to_input(X):
    """
    reshapes data
    """
    max_length = 10
    [Num_examples, dim] = X.shape
    input_X = X[:,2:].reshape(Num_examples, max_length, 66)
    
    return input_X
    

In [45]:
def Random_Selection_train_test (X_all, N):
#    import random
    " This fucntion takes return train and test samples for the LSTM: X dataset, N - number of training samples (i.e persons)"
    Persons = X_all[:,1]
    uniquePersons = np.unique(Persons)
    random.shuffle(uniquePersons)
    num_dim = np.shape(X_all)[1]
    X_train = np.array([], dtype=np.int64).reshape(0,num_dim)
    Y_train = np.array([])
    X_test = np.array([])
    Y_test = np.array([])
    print(X_train.shape)
    for Person in range(1,N):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        #print(X_all[[indexes], :][0,0,:,:].shape)
        Person_X_val = X_all[[indexes], :][0,0,:,:]
        X_train= np.append(X_train,Person_X_val)
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_train = np.append(Y_train, X_all[[indexes], 0])
    X_train=X_train.reshape(Y_train.size, num_dim)    
    for Person in range(N,uniquePersons.size):
        indexes = np.where(X_all[:,1]==uniquePersons[Person])
        X_test = np.append(X_test, X_all[[indexes], :][0,0,:,:])
        label =  np.full((indexes[0].size),uniquePersons[Person])
        Y_test =  np.append(Y_test, X_all[[indexes], 0])
    X_test=X_test.reshape(Y_test.size, num_dim)       
    return X_train, Y_train, X_test, Y_test
            
            


In [46]:
def convert_to_one_hot(inputY, C):
    N= inputY.size
    Y=np.zeros((N,C))
    for i in range (0, inputY.size):
        Y[i, int(inputY[i]-1)] = 1
        
    
    return Y


In [65]:
X_train, Y_train, X_test, Y_test = Random_Selection_train_test (X_all.values, 12)

(0, 662)


### 2.2 Keras and mini-batching 

In this exercise, we want to train Keras using mini-batches. However, most deep learning frameworks require that all sequences in the same mini-batch have the same length. This is what allows vectorization to work: If you had a 3-word sentence and a 4-word sentence, then the computations needed for them are different (one takes 3 steps of an LSTM, one takes 4 steps) so it's just not possible to do them both at the same time.

The common solution to this is to use padding. Specifically, set a maximum sequence length, and pad all sequences to the same length. For example, of the maximum sequence length is 20, we could pad every sentence with "0"s so that each input sentence is of length 20. Thus, a sentence "i love you" would be represented as $(e_{i}, e_{love}, e_{you}, \vec{0}, \vec{0}, \ldots, \vec{0})$. In this example, any sentences longer than 20 words would have to be truncated. One simple way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set. 


In [79]:
def Gait_model(data):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input - data

    Returns:
    model -- a model instance in Keras
    """
   
    ### START CODE HERE ###
    gait_data = Input(data.shape, dtype='float32')
    print(gait_data.shape)
    #gait_data = tf.keras.backend.squeeze(gait_data, 0)
    #examples = gait_data
    #max_length = 15
    #examples = examples[2:,:]
    #print(examples.shape)
    #Num_examples = examples.shape[0]
    #embeddings = examples.reshape(Num_examples, max_length, 144)
    #print(embeddings)
    #embeddings = X_train# Reshape_to_input(X_train)
    #embeddings = tf.convert_to_tensor(embeddings, name="raw_data", dtype=tf.float32)

    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    print(gait_data.shape)
    X = LSTM(128, return_sequences=True)( gait_data)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(3)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=gait_data, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [73]:
# prepare the data
X_train = Reshape_to_input(X_train)
X_test = Reshape_to_input(X_test)

ValueError: too many values to unpack (expected 2)

Run the following cell to create your model and check its summary. Because all sentences in the dataset are less than 10 words, we chose `max_len = 10`.  You should see your architecture, it uses "20,223,927" parameters, of which 20,000,050 (the word embeddings) are non-trainable, and the remaining 223,877 are. Because our vocabulary size has 400,001 words (with valid indices from 0 to 400,000) there are 400,001\*50 = 20,000,050 non-trainable parameters. 

In [80]:
maxLen = 10
print(X_train.shape)
model = Gait_model((X_train[0,:,:]))
model.summary()

(267, 10, 66)
(?, 10, 66)
(?, 10, 66)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 10, 66)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 10, 128)           99840     
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 387       
_________________________________________________________________
activation_5 (Activation)    (None, 3)

As usual, after creating your model in Keras, you need to compile it and define what loss, optimizer and metrics your are want to use. Compile your model using `categorical_crossentropy` loss, `adam` optimizer and `['accuracy']` metrics:

In [81]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

It's time to train your model. Your Emojifier-V2 `model` takes as input an array of shape (`m`, `max_len`) and outputs probability vectors of shape (`m`, `number of classes`). We thus have to convert X_train (array of sentences as strings) to X_train_indices (array of sentences as list of word indices), and Y_train (labels as indices) to Y_train_oh (labels as one-hot vectors).

In [76]:
#X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 3)
Y_test_oh = convert_to_one_hot(Y_test, C = 3)

Fit the Keras model on `X_train_indices` and `Y_train_oh`. We will use `epochs = 50` and `batch_size = 32`.

In [91]:
model.fit(X_train, Y_train_oh, epochs = 12, batch_size = 15, shuffle=True)

Epoch 1/12
262/262 [==============================] - 2s 7ms/step - loss: 0.0024 - acc: 1.0000
Epoch 2/12
262/262 [==============================] - 1s 5ms/step - loss: 0.0015 - acc: 1.0000
Epoch 3/12
262/262 [==============================] - 1s 3ms/step - loss: 0.0026 - acc: 1.0000
Epoch 4/12
262/262 [==============================] - 1s 3ms/step - loss: 9.9687e-04 - acc: 1.0000
Epoch 5/12
262/262 [==============================] - 1s 2ms/step - loss: 5.8223e-04 - acc: 1.0000
Epoch 6/12
262/262 [==============================] - 1s 2ms/step - loss: 7.2041e-04 - acc: 1.0000
Epoch 7/12
262/262 [==============================] - 1s 2ms/step - loss: 5.5999e-04 - acc: 1.0000
Epoch 8/12
262/262 [==============================] - 1s 2ms/step - loss: 0.0011 - acc: 1.0000
Epoch 9/12
262/262 [==============================] - 1s 2ms/step - loss: 5.8533e-04 - acc: 1.0000
Epoch 10/12
262/262 [==============================] - 1s 2ms/step - loss: 6.8329e-04 - acc: 1.0000
Epoch 11/12
262/262 [====

Your model should perform close to **100% accuracy** on the training set. The exact accuracy you get may be a little different. Run the following cell to evaluate your model on the test set. 

In [92]:
#X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
#Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test, Y_test_oh)
print()
print("Test accuracy = ", acc)

190/190 [==============================] - 0s 1ms/step

Test accuracy =  0.95789473747


In [89]:
trials = 5
Accuracy = X_train = np.zeros(shape=(trials))

for i in range(0,trials):
    X_train, Y_train, X_test, Y_test = Random_Selection_train_test (X_all.values, 12)
    Y_train_oh = convert_to_one_hot(Y_train, C = 3)
    Y_test_oh = convert_to_one_hot(Y_test, C = 3)
    X_train = Reshape_to_input(X_train)
    X_test = Reshape_to_input(X_test)
    model.fit(X_train, Y_train_oh, epochs = 12, batch_size = 15, shuffle=True)
    loss, acc = model.evaluate(X_test, Y_test_oh)
    Accuracy[i]=acc
    
print(Accuracy)    

[ 0.  0.  0.  0.  0.]
(0, 662)
Epoch 1/12
265/265 [==============================] - 2s 6ms/step - loss: 0.0018 - acc: 1.0000
Epoch 2/12
265/265 [==============================] - 1s 4ms/step - loss: 4.2743e-04 - acc: 1.0000
Epoch 3/12
265/265 [==============================] - 1s 3ms/step - loss: 8.0727e-04 - acc: 1.0000
Epoch 4/12
265/265 [==============================] - 1s 3ms/step - loss: 5.7747e-04 - acc: 1.0000
Epoch 5/12
265/265 [==============================] - 0s 2ms/step - loss: 3.3361e-04 - acc: 1.0000
Epoch 6/12
265/265 [==============================] - 1s 2ms/step - loss: 2.6743e-04 - acc: 1.0000
Epoch 7/12
265/265 [==============================] - 0s 2ms/step - loss: 2.0896e-04 - acc: 1.0000
Epoch 8/12
265/265 [==============================] - 1s 2ms/step - loss: 1.7914e-04 - acc: 1.0000
Epoch 9/12
265/265 [==============================] - 1s 2ms/step - loss: 2.0732e-04 - acc: 1.0000
Epoch 10/12
265/265 [==============================] - 1s 2ms/step - loss: 1.5778e

In [90]:

print(Accuracy)  


[ 0.99497487  1.          0.97237569  0.99447514  0.9631579 ]


You should get a test accuracy between 80% and 95%. Run the cell below to see the mislabelled examples. 